### Dataset Download

In [1]:
import gdown

gdown.download(id="1ZEyNMEO43u3qhJAwJeBZxFBEYc_pVYZQ")

!unzip dataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=1ZEyNMEO43u3qhJAwJeBZxFBEYc_pVYZQ
From (redirected): https://drive.google.com/uc?id=1ZEyNMEO43u3qhJAwJeBZxFBEYc_pVYZQ&confirm=t&uuid=fa2e7b78-a34a-46ff-ba78-2d8ca50b99e0
To: /content/dataset.zip
100%|██████████| 1.13G/1.13G [00:14<00:00, 79.3MB/s]


Streaming output truncated to the last 5000 lines.
  inflating: dataset/val/no/531.npy  
  inflating: dataset/val/no/257.npy  
  inflating: dataset/val/no/243.npy  
  inflating: dataset/val/no/525.npy  
  inflating: dataset/val/no/1099.npy  
  inflating: dataset/val/no/1927.npy  
  inflating: dataset/val/no/1933.npy  
  inflating: dataset/val/no/519.npy  
  inflating: dataset/val/no/1066.npy  
  inflating: dataset/val/no/1700.npy  
  inflating: dataset/val/no/294.npy  
  inflating: dataset/val/no/2209.npy  
  inflating: dataset/val/no/280.npy  
  inflating: dataset/val/no/1714.npy  
  inflating: dataset/val/no/1072.npy  
  inflating: dataset/val/no/2235.npy  
  inflating: dataset/val/no/1728.npy  
  inflating: dataset/val/no/2221.npy  
  inflating: dataset/val/no/733.npy  
  inflating: dataset/val/no/727.npy  
  inflating: dataset/val/no/1502.npy  
  inflating: dataset/val/no/1264.npy  
  inflating: dataset/val/no/928.npy  
  inflating: dataset/val/no/1270.npy  
  inflating: dataset/va

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### **Dataset Preparation**  

The dataset consists of `.npy` files stored in class-specific directories. A custom `NPZDataset` class is used to load and preprocess the data:  

- **Class Mapping:** Three categories—`no (0)`, `sphere (1)`, and `vort (2)`.  
- **Loading Images:** Each `.npy` file is loaded as a NumPy array and converted into a PyTorch tensor.  
- **Channel Expansion:** Since the images are single-channel, they are repeated across three channels for compatibility with the pretrained CNN models.  
- **DataLoader Setup:**  
  - **Training Data:** Batch size of `128`, shuffled for better generalization.  
  - **Validation Data:** Batch size of `32`, no shuffling to maintain consistency.  

This ensures efficient loading and preprocessing for model training.

In [2]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF

class NPZDataset(Dataset):
    def __init__(self, root_dir):
        self.data = []
        self.labels = []
        self.class_map = {'no': 0, 'sphere': 1, 'vort': 2}

        for class_name in self.class_map.keys():
            class_dir = os.path.join(root_dir, class_name)
            for file in os.listdir(class_dir):
                if file.endswith(".npy"):
                    self.data.append(os.path.join(class_dir, file))
                    self.labels.append(self.class_map[class_name])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = np.load(self.data[idx])

        # Convert to tensor and normalize
        image = torch.tensor(image, dtype=torch.float32,device=device)

        image = image.repeat(3, 1, 1)

        label = torch.tensor(self.labels[idx], dtype=torch.long , device=device)
        return image, label

# Load dataset
data_path = "./dataset"
train_dataset = NPZDataset(os.path.join(data_path, "train"))
val_dataset = NPZDataset(os.path.join(data_path, "val"))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### **Model Selection and Modification**  
This section loads the **pretrained MobileNet** model and modifies it for the task of classifying gravitational lensing images.

#### **Key Steps:**  

1. **Model Selection:**  
   - Uses **MobileNetV3-Large**, a lightweight convolutional neural network optimized for efficiency.

2. **Modifying the Classifier:**  
   - The original classifier is designed for ImageNet (1000 classes).  
   - The final fully connected layer (`model.classifier[3]`) is replaced with a new `nn.Linear` layer:

3. **Loading Pretrained Weights:**  
   - Loads weights from `Common_Test_Model.pth`, which contains parameters from the previously trained model.

This setup allows the model to leverage pretrained MobileNetV3 features while adapting to the specific task of classifying gravitational lensing images.

In [ ]:
from torchvision.io import decode_image
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights

weights = MobileNet_V3_Large_Weights.DEFAULT
model = mobilenet_v3_large(weights=weights)

model.classifier[3] = nn.Linear(in_features=1280, out_features=3, bias=True)

pretrained_model = model.to(device)

model.load_state_dict(torch.load('Common_Test_Model.pth'))

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import roc_curve, auc

### **Physics-Informed Neural Network (PINN) Implementation**  
This section defines the **PINN architecture** for classifying gravitational lensing images by integrating physical constraints into the neural network.

#### **Key Components of the PINN:**  

1. **Encoder (Feature Extraction):**  
   - Uses the **pretrained MobileNet** (`pretrained_model.features`) to extract feature representations.  
   - Freezes its parameters (`requires_grad_(False)`) to leverage pretrained common test knowledge.

2. **Decoder (Lens Potential Prediction - ψ):**  
   - Uses **transposed convolutions** to upsample encoded features to predict the lens potential ψ.  
   - Outputs a tensor of shape `[batch, 1, h, w]`.  

3. **Kappa Computation (Physics Integration):**  
   - Computes **convergence κ** using the discrete **Laplacian operator**:  
  $$
  \kappa = \frac{1}{2} \nabla^2 \psi
  $$  
   - This ensures the network respects the physical relationship between ψ and κ.  

4. **Kappa CNN (Feature Extraction from κ):**  
   - A convolutional subnetwork processes κ to extract **spatial features** for classification.  

5. **Classifier (Final Prediction):**  
   - Combines **image features (from the encoder)** and **κ features**.  
   - Passes the concatenated vector through fully connected layers to output logits for the three classes.  

---

### **Loss Functions**  
The model optimizes a **combined loss** to balance classification and physical constraints:  

1. **Classification Loss** (`loss_class`) → **Cross-entropy loss** for multi-class classification.  
2. **Smoothness Loss** (`loss_smooth`) → Encourages **spatial smoothness in ψ** by penalizing large gradients.  
3. **Physics Loss** (`loss_phys`) → Enforces **κ ≥ 0** using a ReLU penalty on negative κ values.  

$$
\text{loss}_{\text{total}} = \text{loss}_{\text{class}} + \lambda_{\text{smooth}} \cdot \text{loss}_{\text{smooth}} + \lambda_{\text{phys}} \cdot \text{loss}_{\text{phys}}
$$

This **hybrid approach** integrates **deep learning with gravitational lensing physics**, improving classification performance while ensuring physical consistency.

In [6]:
# Define the PINN class
class PINN(nn.Module):
    def __init__(self, num_classes=3):
        super(PINN, self).__init__()

        self.encoder = pretrained_model.features
        self.encoder.requires_grad_(False)
        self.encoder.eval()

        # Decoder: Upsamples features to predict psi
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(960, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1),
        )

        # Kappa CNN: Processes kappa to extract features
        self.kappa_cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64, 32),
        )

        # Classifier: Combines image and kappa features for classification
        self.classifier = nn.Sequential(
            nn.Linear(960 + 32, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        # Extract features from the encoder
        features = self.encoder(x)

        # Predict the lens potential psi
        psi = self.decoder(features)

        # Compute kappa from psi using the Laplacian
        kappa = self.compute_kappa(psi)

        # Extract features from kappa
        kappa_features = self.kappa_cnn(kappa)

        # Pool image features and combine with kappa features
        image_features = F.adaptive_avg_pool2d(features, (1, 1)).flatten(1)
        combined_features = torch.cat([image_features, kappa_features], dim=1)

        # Predict class logits
        class_logits = self.classifier(combined_features)

        return class_logits, psi, kappa

    def compute_kappa(self, psi):
        # Compute the discrete Laplacian using finite differences
        psi_padded = F.pad(psi, (1, 1, 1, 1), mode='replicate')
        laplacian = (psi_padded[:, :, 2:, 1:-1] + psi_padded[:, :, :-2, 1:-1] +
                     psi_padded[:, :, 1:-1, 2:] + psi_padded[:, :, 1:-1, :-2] -
                     4 * psi)
        kappa = 0.5 * laplacian  # Convergence kappa, proportional to the Laplacian
        return kappa

# Smoothness loss for psi
def smoothness_loss(psi):
    dx = psi[:, :, :, 1:] - psi[:, :, :, :-1]  # Gradient in x-direction
    dy = psi[:, :, 1:, :] - psi[:, :, :-1, :]  # Gradient in y-direction
    return (dx.pow(2).mean() + dy.pow(2).mean())

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

### **Training and Validation**  
- **Model Setup:** Uses PINN with `Adam (lr=0.001)`, `CrossEntropyLoss`, and runs on the GPU.
- **Training:**  
  - Forward pass → Computes **class logits (`ψ, κ`)**.  
  - **Losses:**  
    - `loss_class`: Cross-entropy (classification).  
    - `loss_smooth`: Penalizes sharp variations in `ψ`.  
    - `loss_phys`: Enforces **κ ≥ 0**.  
    - `loss_total = loss_class + loss_smooth + loss_phys`.  
  - Backpropagation (`loss.backward()`, `optimizer.step()`).  
- **Validation:**  
  - Computes losses and accuracy.  
  - **Accuracy = (correct predictions / total) × 100**.  
- **Output:** Per-epoch train/validation losses and accuracy.

In [8]:
# Initialize model, optimizer, and loss function
model = PINN(num_classes=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10
lambda_phys = 0.1  # Weight for physics loss

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_loss_class = 0.0
    running_loss_smooth = 0.0
    running_loss_phys = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        class_logits, psi, kappa = model(images)

        # Compute losses
        loss_class = criterion(class_logits, labels)  # Classification loss
        loss_smooth = smoothness_loss(psi)            # Smoothness loss on psi
        loss_phys = torch.mean(torch.relu(-kappa))    # Physics loss: penalize negative kappa

        # Total loss
        loss = loss_class + loss_smooth + loss_phys

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_loss_class += loss_class.item()
        running_loss_smooth += loss_smooth.item()
        running_loss_phys += loss_phys.item()

    model.eval()
    val_loss = 0.0
    val_loss_class = 0.0
    val_loss_smooth = 0.0
    val_loss_phys = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            class_logits, psi, kappa = model(images)

            # Compute losses
            loss_class = criterion(class_logits, labels)  # Classification loss
            loss_smooth = smoothness_loss(psi)            # Smoothness loss on psi
            loss_phys = torch.mean(torch.relu(-kappa))    # Physics loss: penalize negative kappa

            # Total loss
            loss = loss_class + loss_smooth + loss_phys

            val_loss += loss.item()
            val_loss_class += loss_class.item()
            val_loss_smooth += loss_smooth.item()
            val_loss_phys += loss_phys.item()

            _, predicted = torch.max(class_logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Loss (Class): {running_loss_class / len(train_loader):.4f}, '
          f'Train Loss (Smooth): {running_loss_smooth / len(train_loader):.4f}, '
          f'Train Loss (Phys): {running_loss_phys / len(train_loader):.4f}, '
          f'Val Loss: {val_loss / len(val_loader):.4f}, '
          f'Val Loss (Class): {val_loss_class / len(val_loader):.4f}, '
          f'Val Loss (Smooth): {val_loss_smooth / len(val_loader):.4f}, '
          f'Val Loss (Phys): {val_loss_phys / len(val_loader):.4f}, '
          f'Val Accuracy: {100 * correct / total:.2f}%')

Epoch 1, Train Loss: 0.2401, Train Loss (Class): 0.2362, Train Loss (Smooth): 0.0010, Train Loss (Phys): 0.0029, Val Loss: 0.2541, Val Loss (Class): 0.2537, Val Loss (Smooth): 0.0000, Val Loss (Phys): 0.0004, Val Accuracy: 91.04%
Epoch 2, Train Loss: 0.1959, Train Loss (Class): 0.1954, Train Loss (Smooth): 0.0000, Train Loss (Phys): 0.0004, Val Loss: 0.2458, Val Loss (Class): 0.2457, Val Loss (Smooth): 0.0000, Val Loss (Phys): 0.0002, Val Accuracy: 91.07%
Epoch 3, Train Loss: 0.1935, Train Loss (Class): 0.1932, Train Loss (Smooth): 0.0000, Train Loss (Phys): 0.0003, Val Loss: 0.2442, Val Loss (Class): 0.2433, Val Loss (Smooth): 0.0000, Val Loss (Phys): 0.0009, Val Accuracy: 90.83%
Epoch 4, Train Loss: 0.1943, Train Loss (Class): 0.1940, Train Loss (Smooth): 0.0000, Train Loss (Phys): 0.0004, Val Loss: 0.2459, Val Loss (Class): 0.2457, Val Loss (Smooth): 0.0000, Val Loss (Phys): 0.0002, Val Accuracy: 91.15%
Epoch 5, Train Loss: 0.1908, Train Loss (Class): 0.1905, Train Loss (Smooth): 0.

In [16]:
# Evaluate model
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in val_loader:
        outputs,_,_ = model(images)
        probabilities = torch.softmax(outputs[:, :3], dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(probabilities.cpu().numpy())

# Calculate AUC score
auc_score = roc_auc_score(y_true, y_pred, multi_class='ovr')
print(f"AUC Score: {auc_score:.4f}")

AUC Score: 0.9821


In [14]:
import numpy as np
import torch
import torch.nn.functional as F
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc
from itertools import cycle

# Evaluation with ROC and AUC
model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        class_logits, _, _ = model(images)
        probs = F.softmax(class_logits, dim=1)
        all_probs.append(probs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

all_probs = np.concatenate(all_probs)
all_labels = np.concatenate(all_labels)

# Compute ROC curve and AUC for each class
n_classes = all_probs.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(all_labels == i, all_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Create Plotly figure
fig = go.Figure()

colors = cycle(['blue', 'red', 'green', 'purple', 'orange'])  # Extend for more classes

for i, color in zip(range(n_classes), colors):
    fig.add_trace(go.Scatter(
        x=fpr[i], y=tpr[i],
        mode='lines',
        name=f'Class {i} (AUC = {roc_auc[i]:.4f})',
        line=dict(color=color)
    ))

# Add diagonal reference line
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    line=dict(dash='dash', color='black'),
    name='Random (AUC = 0.5)'
))

# Update layout
fig.update_layout(
    title="ROC Curves",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    legend=dict(x=0.7, y=0.1),
)

fig.show()


In [17]:
torch.save(model.state_dict(), 'Test_V_Model.pth')